# Variation des Magnetfeldes, um Asymmetrie verschwinden zu lassen.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from STEP import STEP

In [4]:
ebins = np.array([  0.98 ,   2.144,   2.336,   2.544,   2.784,   3.04 ,   3.312,
         3.6  ,   3.92 ,   4.288,   4.672,   5.088,   5.568,   6.08 ,
         6.624,   7.2  ,   7.84 ,   8.576,   9.344,  10.176,  11.136,
        12.16 ,  13.248,  14.4  ,  15.68 ,  17.152,  18.688,  20.352,
        22.272,  24.32 ,  26.496,  28.8  ,  31.36 ,  34.304,  37.376,
        40.704,  44.544,  48.64 ,  52.992,  57.6  ,  62.72 ,  68.608,
        74.752,  81.408,  89.088,  97.28 , 105.984, 115.2  , 125.44 ,
       137.216, 149.504, 162.816, 178.176, 194.56 , 211.968, 230.4  ,
       372.736])
def grenz(t):
    return -0.5*t + 20

dat = STEP(2021, 12, 4, rpath='data/STEP/', mag_path='data/mag/srf', mag_frame = 'srf')
# dat = STEP(2021,12,4,mag_path='default',mag_frame='srf')
period =(dt.datetime(2021,12,4,13,50),dt.datetime(2021,12,4,14,30))

STEP-Data loaded successfully.
STEP-Data combined successfully.
2021-12-04 00:00:00 data/mag/srf/2021/solo_L2_mag-srf-normal_20211204_V01.cdf


In [5]:
def pw(flow,B,B_offset):
    '''Übergebe den particle flow-Vektor als Geschwindigkeit und den Magnetfeldvektor (am besten in SRF) und berechne die Pitchwinkel über das Skalarprodukt.
    Zusätzlich kann für die Magnetfeldkomponenten ein konstanter Offset übergeben werden.'''
    len_flow = np.sqrt(flow[0]**2 + flow[1]**2 + flow[2]**2)
    len_B = np.sqrt((B[0]+B_offset[0])**2 + (B[1]+B_offset[1])**2 + (B[2]+B_offset[2])**2)
    argument = (flow[0]*(B[0]+B_offset[0]) + flow[1]*(B[1]+B_offset[1]) + flow[2]*(B[2]+B_offset[2]))/len_flow/len_B
    result = np.arccos(argument)
    return result
        
def calc_pw(dat,B_offset):
    '''Berechne die Pitchwinkel für die Elektronen, welche auf STEP treffen in erster Näherung.
    Dafür wird der Winkel zwischen dem particle flow vector der Pixel und dem Magnetfeld herangezogen.
    Kann wieder einen Offset für das Magnetfeld übergeben.'''
    pws =  []
    for i in range(15):
        pws.append(pw(dat.flow_vector[i],[dat.B_R,dat.B_T,dat.B_N],B_offset))
    return np.array(pws)

def average_pw(dat,period,pitchangles,window_width=5):
    '''Berechnung der gemittelten Pitchwinkel'''
    # Maske, da ich nur die Magnetfelddaten innerhalb von period brauche:
    mask = (dat.time > period[0]) * (dat.time <= period[1])
    pw = [[] for i in range(15)]
    pw_time = []
        
    i = 0
    while (period[0] + dt.timedelta(minutes=(i+1)*window_width)) <= period[1]:
        pw_time.append(period[0] + dt.timedelta(minutes=(i+0.5)*window_width))
            
        for k in [i for i in range(1,16)]:
            # Mittelung der Pitchwinkel (k-1, da ich keine Zeit im array stehen habe)
            pw_data = pitchangles[k-1][mask]
            new_pw = np.sum(pw_data[i*window_width:(i+1)*window_width])/window_width
            pw[k-1].append(new_pw)
        i +=1
    return pw, pw_time

In [6]:
def step_plot_correction_manipulated(dat, period, grenz, B_offset):
    '''Berechne und Plotte die Korrektur für manipulierte Magnetfelddaten.'''

    pixel_means, pixel_var = dat.calc_energy_means(ebins=ebins,head=-1, period=period, grenzfunktion=grenz, norm='ptmax')
    pitchangles = calc_pw(dat.mag,B_offset)
    pw, pw_time = average_pw(dat.mag,period,pitchangles)

    year = str(period[0].year - 2000)
    if period[0].month < 10:
        month = '0' + str(period[0].month)
    else:
        month = str(period[0].month)
    if period[0].day < 10:
        day = '0' + str(period[0].day)
    else:
        day = str(period[0].day)
    
    fig, ax = dat.step_plot('time', 'mean of energy [keV]', 'energy means with pitch angle correction')
    
    pixel1 = 3
    for pixel2 in range(1,16):
        ax[pixel2].errorbar(pixel_means[0],pixel_means[pixel1],yerr=np.sqrt(pixel_var[pixel1]),marker='x',label=f'mean pixel {pixel1}')
            
        # Übergebe willkürliche Fehler, da ich diese eh nicht brauche.
        energy2_corrected = dat.energy_correction(pixel_means[pixel2],pw[pixel1-1],pw[pixel2-1],2,2)[0]
            
        ax[pixel2].errorbar(pixel_means[0],energy2_corrected,yerr=np.sqrt(pixel_var[pixel2]),marker='x',label=f'mean pixel {pixel2}')
        ax[pixel2].tick_params(axis='x',labelrotation=45)
        ax[pixel2].legend()
    plt.savefig(f'mag_variation/step_plot_total_correction_energy_means_pixel{pixel1}_{year}_{month}_{day}.png')
    plt.close('all')
    

    fig, ax = dat.step_plot('time', 'difference of energy means [keV]', f'difference of corrected energy means to pixel {pixel1}')
        
    for pixel2 in range(1,16):
        # Übergebe willkürliche Fehler, da ich diese eh nicht brauche.
        energy2_corrected = dat.energy_correction(pixel_means[pixel2],pw[pixel1-1],pw[pixel2-1],2,2)[0]
        diff_corrected = energy2_corrected - pixel_means[pixel1]
        
        ax[pixel2].plot(pixel_means[0],diff_corrected,marker='x')
        ax[pixel2].axhline(0,color='tab:red')
            
        ax[pixel2].tick_params(axis='x',labelrotation=45)
    plt.savefig(f'mag_variation/step_plot_total_correction_differences_energy_pixel{pixel1}_{year}_{month}_{day}_offset.png')
    plt.close('all')


def step_plot_correction_multiple_offsets(dat, period, grenz, Offsets, title=None):
    '''Berechne und Plotte die Korrektur für manipulierte Magnetfelddaten und packe alles in einen Plot.'''

    pixel_means, pixel_var = dat.calc_energy_means(ebins=ebins,head=-1, period=period, grenzfunktion=grenz, norm='ptmax')
    
    # Offsets = np.array([np.zeros(11),np.zeros(11),np.array([i for i in range(-5,6)])]).T

    pixel1 = 3
    
    fig, ax = dat.step_plot('time', 'difference of energy means [keV]', f'difference of corrected energy means to pixel {pixel1}')

    for B_offset in Offsets:
        pitchangles = calc_pw(dat.mag,B_offset)
        pw, pw_time = average_pw(dat.mag,period,pitchangles)

        year = str(period[0].year - 2000)
        if period[0].month < 10:
            month = '0' + str(period[0].month)
        else:
            month = str(period[0].month)
        if period[0].day < 10:
            day = '0' + str(period[0].day)
        else:
            day = str(period[0].day)

        ax[0].plot([],[],label=str(B_offset))

        for pixel2 in range(1,16):
            # Übergebe willkürliche Fehler, da ich diese eh nicht brauche.
            energy2_corrected = dat.energy_correction(pixel_means[pixel2],pw[pixel1-1],pw[pixel2-1],2,2)[0]
            diff_corrected = energy2_corrected - pixel_means[pixel1]
            
            ax[pixel2].plot(pixel_means[0],diff_corrected,marker='x')
            ax[pixel2].axhline(0,color='tab:red')
                
            ax[pixel2].tick_params(axis='x',labelrotation=45)
    ax[0].legend()
    if type(title) == str:
        plt.savefig(f'mag_variation/' + title + '.png')
    else:
        plt.savefig(f'mag_variation/step_plot_total_correction_differences_energy_pixel{pixel1}_{year}_{month}_{day}_multiple_offsets.png')
    plt.close('all')

In [7]:
OffsetR = np.array([np.array([i for i in range(-3,4)]),np.zeros(7),np.zeros(7)]).T
OffsetT = np.array([np.zeros(7),np.array([i for i in range(-3,4)]),np.zeros(7)]).T
OffsetN = np.array([np.zeros(7),np.zeros(7),np.array([i for i in range(-3,4)])]).T

step_plot_correction_multiple_offsets(dat,period,grenz,OffsetR,'mag_variation_offsetR')
step_plot_correction_multiple_offsets(dat,period,grenz,OffsetT,'mag_variation_offsetT')
step_plot_correction_multiple_offsets(dat,period,grenz,OffsetN,'mag_variation_offsetN')